In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# My code start

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 1000
EPOCHS = 15

In [ ]:
# Import data
train_data = pd.read_csv('train-data.tsv', sep='\t', names=['label', 'text'])
test_data = pd.read_csv('valid-data.tsv', sep='\t', names=['label', 'text'])

# Map ham span and convert values
train_data['label'] = train_data['label'].map({'ham': 0.0, 'spam': 1.0})
test_data['label'] = test_data['label'].map({'ham': 0.0, 'spam': 1.0})

train_data['label'] = np.asarray(train_data['label']).astype('float32')
test_data['label'] = np.asarray(test_data['label']).astype('float32')

train_data = tf.data.Dataset.from_tensor_slices((train_data['text'], train_data['label']))
test_data = tf.data.Dataset.from_tensor_slices((test_data['text'], test_data['label']))

In [ ]:
# Examine data
#for text, label in train_data.take(1):
#  print('texts: ', text.numpy())
#  print('label: ', label.numpy())

In [ ]:
train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)

encoder.adapt(train_data.map(lambda text, label: text))
vocab = np.array(encoder.get_vocabulary())

In [ ]:
# Build the model
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

assert [layer.supports_masking for layer in model.layers] == [False, True, True, True, True]

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(train_data, epochs=EPOCHS, validation_data=test_data, validation_steps=30, verbose=2)

test_loss, test_acc = model.evaluate(test_data)

In [ ]:
# My code end

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # prediction = model.predict(np.array([pred_text]))[0][0]
  prediction = model.predict(tf.constant([pred_text]))[0][0]

  if round(prediction) >= 1:
    return [prediction, 'spam']
  return[prediction, 'ham']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
